# Notebook for topic modeling 

# 0. Imports

In [2]:
## load packages 
import pandas as pd
import re
import numpy as np

#nltk imports
!pip install nltk # can install on terminal or by uncommenting this line
import nltk; nltk.download('punkt'); nltk.download('stopwords')
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

## sklearn imports
from sklearn.feature_extraction.text import CountVectorizer

## lda
#!pip install gensim # can install by uncommenting this line
from gensim import corpora
import gensim

## visualizing LDA--likely need to install
#!pip install pyLDAvis # can install by uncommenting this line
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()

## print mult things
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## random
import random
import string; punctlist = [char for char in string.punctuation] # list of english punctuation marks

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allisonburg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/allisonburg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 0. Load data

In [3]:
ab = pd.read_csv("../public_data/airbnb_text.zip")
ab.head()

,id,name,name_upper,neighbourhood_group,price
0,2539,Clean & quiet apt home by the park,CLEAN & QUIET APT HOME BY THE PARK,Brooklyn,149
1,2595,Skylit Midtown Castle,SKYLIT MIDTOWN CASTLE,Manhattan,225
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,THE VILLAGE OF HARLEM....NEW YORK !,Manhattan,150
3,3831,Cozy Entire Floor of Brownstone,COZY ENTIRE FLOOR OF BROWNSTONE,Brooklyn,89
4,5022,Entire Apt: Spacious Studio/Loft by central park,ENTIRE APT: SPACIOUS STUDIO/LOFT BY CENTRAL PARK,Manhattan,80


# 1. Preprocess documents

In this case, each name/name_upper, or listing title, we're treating as a document

## 1.1 Load stopwords list and augment with our own custom ones

In [4]:
list_stopwords = stopwords.words("english")

custom_words_toadd = ['apartment', 'new york', 'nyc',
                      'bronx', 'brooklyn',
                     'manhattan', 'queens', 
                      'staten island']

list_stopwords_new = list_stopwords + custom_words_toadd


## 1.2 Remove stopwords from lowercase version of corpus


In [5]:
## convert to lowercase and a list
corpus_lower = ab.name.str.lower().to_list()
corpus_lower[0:5]

## use wordpunct tokenize and filter out with one
example_listing = corpus_lower[3]
nostop_listing = [word for word in wordpunct_tokenize(example_listing) 
                          if word not in list_stopwords_new]
nostop_listing

['clean & quiet apt home by the park',
 'skylit midtown castle',
 'the village of harlem....new york !',
 'cozy entire floor of brownstone',
 'entire apt: spacious studio/loft by central park']

['cozy', 'entire', 'floor', 'brownstone']

## 1.3 stem and remove non-alpha

Other contexts we may want to leave digits in

In [6]:
## initialize stemmer
porter = PorterStemmer()

## apply to one by iterating
## over the tokens in the list
example_listing_preprocess = [porter.stem(token) 
                            for token in nostop_listing 
                            if token.isalpha() and 
                            len(token) > 2]

example_listing_preprocess

['cozi', 'entir', 'floor', 'brownston']

In [7]:
example_listing
example_listing_preprocess

'cozy entire floor of brownstone'

['cozi', 'entir', 'floor', 'brownston']

## 1.4 Activity 1

The above example performed preprocessing on a single Airbnb listing. We want to generalize this preprocessing across all listings.

- Embed step two (remove stopwords) and step three (stem) into one or two functions that take in a raw string (eg the raw text of an Airbnb review) and return a preprocessed string 
- Apply the function iteratively to preprocess all the texts in `corpus_lower`. Output could either be a list where each list element is a string of a list (e.g., `cozy brownstone apt`), or a list of lists where each element is a tokenized string (e.g., `['cozy', 'brownstone', 'apt'])`

Output is flexible: it could be a list of lists containing tokenized/stemmed text or a list of strings.

In [8]:
# your code here to define the function(s)
porter = PorterStemmer()

def process_string(text):
    if pd.isna(text):
        return ""
    lower_text = text.lower()
    ## use wordpunct tokenize and filter out with one
    nostop_listing = [word for word in wordpunct_tokenize(lower_text)
                              if word not in list_stopwords_new]
    ## apply to one by iterating
    ## over the tokens in the list
    text_preprocess = [porter.stem(token)
                                for token in nostop_listing
                                if token.isalpha() and
                                len(token) > 2]
    return text_preprocess
# your code here to apply the function
preprocessing = [process_string(text) for text in corpus_lower]
preprocessing[:6]



# your code here to apply the function

[['clean', 'quiet', 'apt', 'home', 'park'],
 ['skylit', 'midtown', 'castl'],
 ['villag', 'harlem', 'new', 'york'],
 ['cozi', 'entir', 'floor', 'brownston'],
 ['entir', 'apt', 'spaciou', 'studio', 'loft', 'central', 'park'],
 ['larg', 'cozi', 'midtown', 'east']]

# 2. Create a document-term matrix and do some basic diagnostics (more manual approach)

Here we'll create a DTM first using the raw documents; in the activity, you'll create one using the preprocessed docs
that you created in activity 1

## 2.1 Define the dtm function and select data to transform into a document-term matrix

In [9]:
## function provided
def create_dtm(list_of_strings, metadata):
    """ 
    Function to create dense document-term matrix (DTM) from a list of strings and provided metadata. 
    A sparse DTM is a list of term_index/doc_index tuples: if a given term occurs in a given doc at least once, 
        then this count is listed as a tuple; if not, that term/doc pair is omitted. 
    In a dense DTM, each row is one text (e.g., an Airbnb listing), each column is a term, and 
        each cell indicates the frequency of that word in that text. 
    
    Parameters:
        list_of_strings (Series): each row contains a preprocessed string (need not be tokenized)
        metadata (DataFrame): contains document-level covariates
    
    Returns:
        Dense DTM with metadata on left and then one column per word in lexicon
    """
    
    # initialize a sklearn tokenizer; this helps us tokenize the preprocessed string input
    vectorizer = CountVectorizer(lowercase = True) 
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    print('Sparse matrix form:\n', dtm_sparse[:3]) # take a look at sparse representation
    print()
    
    # switch the dataframe from the sparse representation to the normal dense representation (so we can treat it as regular dataframe)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out ())
    print('Dense matrix form:\n', dtm_dense_named.head()) # take a look at dense representation
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1) # add back document-level covariates

    return(dtm_dense_named_withid)

In [10]:

## filter out na's
## for shorter runtime, random sampling of 1000
## get metadata for those
## and also renaming price col since it's likely to be corpus word
ab_small = ab.loc[~ab.name.isnull(),
           ['id', 'neighbourhood_group', 'price', 'name']].copy().rename(columns = {'price':
            'price_rawdata'}).sample(n = 1000, random_state = 422)

ab_small['name_lower'] = ab_small['name'].str.lower()
ab_small.head()

,id,neighbourhood_group,price_rawdata,name,name_lower
23821,19227560,Queens,100,Super Cozy!,super cozy!
22905,18560625,Brooklyn,30,Beautiful Private Bedroom by Prospect Park,beautiful private bedroom by prospect park
20426,16289576,Manhattan,80,Best Location on the Upper West Side! - Part II,best location on the upper west side! - part ii
2018,893413,Manhattan,2500,Architecturally Stunning Former Synagogue!,architecturally stunning former synagogue!
18790,14882137,Queens,50,"Large, beautiful room near Bushwick","large, beautiful room near bushwick"


## 2.2 Execute the dtm function to create the document-term matrix

In [11]:
## example application on raw lowercase texts; 
dtm_nopre = create_dtm(list_of_strings= ab_small.name_lower,
                      metadata = ab_small[['id', 'neighbourhood_group', 'price_rawdata']])



Sparse matrix form:
   (0, 841)	1
  (0, 281)	1
  (1, 152)	1
  (1, 693)	1
  (1, 157)	1
  (1, 205)	1
  (1, 698)	1
  (1, 653)	1
  (2, 165)	1
  (2, 537)	1
  (2, 637)	1
  (2, 856)	1
  (2, 902)	1
  (2, 939)	1
  (2, 774)	1
  (2, 657)	1
  (2, 471)	1

Dense matrix form:
    001  10  10m  10min  10mins  1100  12mins  14  15  15min  ...  yoga  york  \
0    0   0    0      0       0     0       0   0   0      0  ...     0     0   
1    0   0    0      0       0     0       0   0   0      0  ...     0     0   
2    0   0    0      0       0     0       0   0   0      0  ...     0     0   
3    0   0    0      0       0     0       0   0   0      0  ...     0     0   
4    0   0    0      0       0     0       0   0   0      0  ...     0     0   

   you  your  yu  zen  ღღღsteps  法拉盛中心私人房間獨立衛浴  溫馨大套房  獨一無二的紐約閣樓  
0    0     0   0    0         0              0      0          0  
1    0     0   0    0         0              0      0          0  
2    0     0   0    0         0              0      0  

In [12]:
## show first set of rows/cols
dtm_nopre.head()

## show arbitrary later cols in resulting data
dtm_nopre.shape
dtm_nopre.iloc[0:5, 480:500]

,index,id,neighbourhood_group,price_rawdata,001,10,10m,10min,10mins,1100,...,yoga,york,you,your,yu,zen,ღღღsteps,法拉盛中心私人房間獨立衛浴,溫馨大套房,獨一無二的紐約閣樓
0,23821,19227560,Queens,100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22905,18560625,Brooklyn,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20426,16289576,Manhattan,80,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018,893413,Manhattan,2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18790,14882137,Queens,50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(1000, 974)

,inclusive,incredible,incredibly,indoor,inn,inq,insane,int,interior,international,interns,invincible,inviting,inwood,island,it,italy,its,jefferson,jewel
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 2.3 Use that matrix/column sums to get basic summary stats of top words

In [13]:
## summing each col
top_terms = dtm_nopre[dtm_nopre.columns[4:]].sum(axis = 0)

## sorting from most frequent to least frequent
top_terms.sort_values(ascending = False)

in           367
room         244
private      163
bedroom      152
apartment    130
            ... 
24hr           1
16             1
1940           1
19th           1
fulton         1
Length: 970, dtype: int64

## 2.4 Activity 2: repeat the above but using the preprocessed text data

- Stick with the same random sample of 1000 `ab_small`
- Apply the preprocessing steps from activity 1 to create a new column in `ab_small` with the preprocessed text (if you got stuck on that, try just removing stopwords)
- Use the `create_dtm` function to create a document-term matrix from the preprocessed data
- Use colsums to summarize

In [14]:
ab_small

,id,neighbourhood_group,price_rawdata,name,name_lower
23821,19227560,Queens,100,Super Cozy!,super cozy!
22905,18560625,Brooklyn,30,Beautiful Private Bedroom by Prospect Park,beautiful private bedroom by prospect park
20426,16289576,Manhattan,80,Best Location on the Upper West Side! - Part II,best location on the upper west side! - part ii
2018,893413,Manhattan,2500,Architecturally Stunning Former Synagogue!,architecturally stunning former synagogue!
18790,14882137,Queens,50,"Large, beautiful room near Bushwick","large, beautiful room near bushwick"
...,...,...,...,...,...
33473,26463879,Brooklyn,65,New york Multi-unit building,new york multi-unit building
12905,9823085,Brooklyn,65,Private room in bright apartment!,private room in bright apartment!
19158,15233387,Manhattan,300,#Bluebird Elegant 1-BR Apartment Near Times Sq...,#bluebird elegant 1-br apartment near times sq...
18451,14520743,Manhattan,95,SPACIOUS room TIME SQUARE,spacious room time square


In [15]:
## your code here 

preprocessed_txt = [process_string(text) for text in ab_small.name]

ab_small["preprocessed_txt"] = " ".join(str(x) for x in preprocessed_txt)
ab_small
dtm_nopre = create_dtm(list_of_strings= ab_small.preprocessed_txt,
                      metadata = ab_small[['id', 'neighbourhood_group', 'price_rawdata']])


# dtm_prep = [create_dtm(list_of_strings = list1, metadata = ab /_small[['id', 'neighbourhood_group', 'price_rawdata']]) for list1 in ab_small.preprocessed_txt]

dtm_prep.colsums()


,id,neighbourhood_group,price_rawdata,name,name_lower,preprocessed_txt
23821,19227560,Queens,100,Super Cozy!,super cozy!,"['super', 'cozi'] ['beauti', 'privat', 'bedroo..."
22905,18560625,Brooklyn,30,Beautiful Private Bedroom by Prospect Park,beautiful private bedroom by prospect park,"['super', 'cozi'] ['beauti', 'privat', 'bedroo..."
20426,16289576,Manhattan,80,Best Location on the Upper West Side! - Part II,best location on the upper west side! - part ii,"['super', 'cozi'] ['beauti', 'privat', 'bedroo..."
2018,893413,Manhattan,2500,Architecturally Stunning Former Synagogue!,architecturally stunning former synagogue!,"['super', 'cozi'] ['beauti', 'privat', 'bedroo..."
18790,14882137,Queens,50,"Large, beautiful room near Bushwick","large, beautiful room near bushwick","['super', 'cozi'] ['beauti', 'privat', 'bedroo..."
...,...,...,...,...,...,...
33473,26463879,Brooklyn,65,New york Multi-unit building,new york multi-unit building,"['super', 'cozi'] ['beauti', 'privat', 'bedroo..."
12905,9823085,Brooklyn,65,Private room in bright apartment!,private room in bright apartment!,"['super', 'cozi'] ['beauti', 'privat', 'bedroo..."
19158,15233387,Manhattan,300,#Bluebird Elegant 1-BR Apartment Near Times Sq...,#bluebird elegant 1-br apartment near times sq...,"['super', 'cozi'] ['beauti', 'privat', 'bedroo..."
18451,14520743,Manhattan,95,SPACIOUS room TIME SQUARE,spacious room time square,"['super', 'cozi'] ['beauti', 'privat', 'bedroo..."


Sparse matrix form:
   (0, 646)	6
  (0, 170)	97
  (0, 60)	60
  (0, 516)	165
  (0, 65)	159
  (0, 519)	13
  (0, 480)	73
  (0, 70)	17
  (0, 388)	32
  (0, 697)	36
  (0, 729)	35
  (0, 584)	31
  (0, 482)	1
  (0, 26)	1
  (0, 629)	7
  (0, 262)	1
  (0, 653)	1
  (0, 367)	64
  (0, 559)	246
  (0, 451)	55
  (0, 102)	28
  (0, 62)	40
  (0, 52)	15
  (0, 119)	42
  (0, 333)	1
  :	:
  (2, 387)	1
  (2, 231)	1
  (2, 6)	1
  (2, 5)	1
  (2, 216)	1
  (2, 640)	1
  (2, 491)	1
  (2, 287)	1
  (2, 347)	1
  (2, 97)	1
  (2, 568)	1
  (2, 188)	1
  (2, 485)	1
  (2, 322)	1
  (2, 454)	1
  (2, 643)	1
  (2, 535)	1
  (2, 404)	1
  (2, 334)	1
  (2, 143)	1
  (2, 739)	1
  (2, 666)	1
  (2, 538)	1
  (2, 77)	1
  (2, 650)	1

Dense matrix form:
    abcd  abod  access  acidot  acogedor  across  address  ador  aesthet  \
0     1     2       1       1         1       1        1     1        1   
1     1     2       1       1         1       1        1     1        1   
2     1     2       1       1         1       1        1     1      

NameError: name 'dtm_prep' is not defined

# 3. Use gensim to more automatically preprocess/estimate a topic model

## 3.1 Creating the objects to feed the LDA modeling function

Different outputs described below: 
- Tokenized and preprocessed text 
- Dictionary 
- Corpus 

In [ ]:

## Step 1: re-tokenize and store in list
## here, i'm doing with the raw random sample of text
## in activity, you should do with the preprocessed texts
text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  ab_small.name_lower]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

### explore first few keys and values
### see that key is just an arbitrary counter; value is the word itself
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]}


## Step 3: filter out very rare and very common words
## here, i'm using the threshold that a word needs to appear in at least
## 5% of docs but not more than 95%
## this is an integer count of docs so i round
lower_bound = round(ab_small.shape[0]*0.05)
upper_bound = round(ab_small.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)
print(f'Filtering out very rare and very common words reduced the \
length of dictionary from {str(raw_len)} to {str(len(text_raw_dict))}.')
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]} # show first five entries after filtering


## Step 4: apply dictionary to TOKENIZED texts
## this creates a mapping between each word 
## in a specific listing and the key in the dictionary.
## for words that remain in the filtered dictionary,
## output is a list where len(list) == n documents
## and each element in the list is a list of tuples
## containing the mappings
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

### can apply doc2bow(one_text, return_missing = True) to print words
### eliminated from the listing bc they're not in filtered dictionary.
### but feeding that one with missing values to
### the lda function can cause errors
corpus_fromdict_showmiss = [text_raw_dict.doc2bow(one_text, return_missing = True)
                            for one_text in text_raw_tokens]
print('Sample of documents represented in dictionary format (with omitted words noted):')
corpus_fromdict_showmiss[:10]

##  3.2 Estimating the model

In [ ]:
## Step 5: we're finally ready to estimate the model!
## full documentation here - https://radimrehurek.com/gensim/models/ldamodel.html
## here, we're feeding the lda function:
## (1) the corpus we created from the dictionary,
## (2) a parameter we decide on for the number of topics (k),
## (3) the dictionary itself,
## (4) parameter for number of passes through training data (more means slower), and
## (5) parameter that returns, for each word remaining in dict, the topic probabilities.
## see documentation for many other arguments you can vary
ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 5, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True)

print(type(ldamod))



## 3.3  Seeing what topics the estimated model discovers

In [ ]:
## Post-model 1: explore corpus-wide summary of topics
### getting the topics and top words; can retrieve diff top words
topics = ldamod.print_topics(num_words = 10)
for topic in topics:
    print(topic)


In [ ]:
    
## Post-model 2: explore topics associated with each document
### for each item in our original dictionary, get list of topic probabilities
l=[ldamod.get_document_topics(item) for item in corpus_fromdict]
### print result
text_raw_tokens[0:5]
l[0:5]

### Visualizing 

In [ ]:
lda_display = gensimvis.prepare(ldamod, corpus_fromdict, text_raw_dict)
pyLDAvis.display(lda_display)

## 3.4 Activity 3

- Preprocess the texts if you haven't already
- Run the topic model with preprocessed texts
- Play around with other parameters like `n_topics` to find a configuration that produces useful topics

If you get stuck on the preprocessing part, you can use below function and example code for applying it. Then continue as above (start with tokenizing).

In [ ]:
def processtext(row, colname):
    
    string_of_col = str(row[colname])
    try:
        processed_string = " ".join([porter.stem(i.lower()) 
                        for i in wordpunct_tokenize(string_of_col) if 
                        i.lower().isalpha() and len(i) >=3]) #and i not in punctlist]) # optional: remove punctuation too 
        return(processed_string)
    except:
        processed_string = "" # to handle data errors where not actually text
        return(processed_string)

ab_small['text_preprocess'] = ab_small.apply(processtext,
                            axis = 1,
                            args = ["name_lower"])
#
ab_small = ab_small[ab_small.text_preprocess != ""].copy()

In [ ]:
# your code here

text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  ab_small.name_lower]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

### explore first few keys and values
### see that key is just an arbitrary counter; value is the word itself
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]}


## Step 3: filter out very rare and very common words
## here, i'm using the threshold that a word needs to appear in at least
## 5% of docs but not more than 95%
## this is an integer count of docs so i round
lower_bound = round(ab_small.shape[0]*0.05)
upper_bound = round(ab_small.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)
print(f'Filtering out very rare and very common words reduced the \
length of dictionary from {str(raw_len)} to {str(len(text_raw_dict))}.')
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]} # show first five entries after filtering


## Step 4: apply dictionary to TOKENIZED texts
## this creates a mapping between each word 
## in a specific listing and the key in the dictionary.
## for words that remain in the filtered dictionary,
## output is a list where len(list) == n documents
## and each element in the list is a list of tuples
## containing the mappings
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

### can apply doc2bow(one_text, return_missing = True) to print words
### eliminated from the listing bc they're not in filtered dictionary.
### but feeding that one with missing values to
### the lda function can cause errors
corpus_fromdict_showmiss = [text_raw_dict.doc2bow(one_text, return_missing = True)
                            for one_text in text_raw_tokens]
print('Sample of documents represented in dictionary format (with omitted words noted):')
corpus_fromdict_showmiss[:10]

ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 5, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True)
topics = ldamod.print_topics(num_words = 10)
for topic in topics:
    print(topic)


lda_display = gensimvis.prepare(ldamod, corpus_fromdict, text_raw_dict)
pyLDAvis.display(lda_display)



In [ ]:
# Hint: example code for preprocessing
def processtext(row, colname):
    
    string_of_col = str(row[colname])
    try:
        processed_string = " ".join([porter.stem(i.lower()) 
                        for i in wordpunct_tokenize(string_of_col) if 
                        i.lower().isalpha() and len(i) >=3]) #and i not in punctlist]) # optional: remove punctuation too 
        return(processed_string)
    except:
        processed_string = "" # to handle data errors where not actually text
        return(processed_string)

ab_small['text_preprocess'] = ab_small.apply(processtext,
                            axis = 1,
                            args = ["name_lower"])
#
ab_small = ab_small[ab_small.text_preprocess != ""].copy()
#
# ab_small.head()